# Get data from canterbury chinese

In [1]:
from docx import Document
import base64
from collections import defaultdict
file_path = '/home/rapids/mounted/finetune_llm/canterbury_chinese.docx'

In [23]:
f = open(file_path, 'rb')
document = Document(f)
f.close()

chinese_full_text = defaultdict(dict)
chapter = '1.1' # this number zero is for prologue
chinese_full_text[chapter] = []

intro_ending_text_to_skip = [
    '而此刻在父執的家園，和斯基亞人奮戰後','戰士乘著用桂冠葉裝飾的戰車等等……',
    '磨坊主的前言', '以下是旅店店主和磨坊主的言語交鋒',
    '封邑管理員的前言',
    '廚師的前言', '廚師故事的前言',
    '律師故事的前言', '某人法律故事的前言',
    '律師的故事', '此處開始律師的故事',
    '律師的引言', '旅舍主人對衆人的說話',
    '巴芙婦人的前言',
    '差役的前言',
    '差役的故事', '差役的故事在此開始',
    '學者的前言', '這裡接續牛津學者故事的前言',
    '商人故事的前言',
    '騎士侍從故事的引言',
    '鄉紳的前言', 
    '賣贖罪卷者故事的引言', '店主對醫生和賣贖罪卷者的説話',
    '賣贖罪卷者的前言', '此處接著是賣贖罪卷者故事的前言',
    '此處開始船長的故事',
    '請看店主對船長和女修院院長高興的說話',
    '女院長故事的前言', '啊主，我們的主',
    '女修院院長的故事','此處開始女修院院長的故事',
    '托拔斯的前言', '請看店主對喬叟説的開心話', 
    '梅里彼的故事', '此處開始喬叟的梅里彼的故事',
    '修道士故事的前言', '店主對修道士説的風趣話', 
    '修女院司鐸故事的前言',
    '第二位修女的前言', 
    '此處開始法典員的鄉農的故事', '第一部份',
    '膳食採購員前言', '以下是採購員故事的前言',
    '堂區主任司鐸的前言', '堂區主任司鐸故事的前言如下。',
    '第一部分完',
    '第二部分接續',
    '請看法庭差役和托缽僧的對話',
    '此處結束巴芙婦人的開場白',
    '托拔斯爵士',
    '第一節',
    '第二節',
    '第三節',
    '～～～～～～～～～～～～～～',
    '～～～～～～～～～～～～',
    '**********************',
    '*******************',
    '***************',
    '*************',
    '************',
    '⋯⋯⋯⋯⋯⋯',
    '⋯⋯⋯⋯⋯',
    '莊主見習騎士兼切肉生對把屁分成十二份的說話',
    '喬叟的後記',
    '第三片段/殘卷',
    '以下是地主管家說的話，和店主回應的話',
    '貪愛錢財乃萬惡的根源：弟茂德前書6。',
    '修道士的故事','此處開始修道士的故事',
    '關於名人的淪喪',
    '路濟弗爾',
    '亞當',
    '叄遜',
    '赫克里斯',
    '尼布查尼撒',
    '巴爾達撒',
    '芝諾比亞',
    '關於卡斯提爾國王彼得',
    '關於賽浦路斯國王彼得',
    '關於倫巴第巴納博子爵',
    '關於烏格林諾，比薩伯爵',
    '尼祿',
    '關於敖羅斐乃',
    '關於亞歷山大',
    '關於尤利烏斯·凱撒',
    '克魯伊塞斯',
    '關於著名的安提約古國王',
    '修女院司鐸故事的尾聲',
    '向瑪利亞祈求',
    '則濟利亞名字的詮釋由熱那亞之',
    '雅各伯弟兄載錄在《黃金傳說》裏',


    '結束',
    '結束。Explicit',
    '騎士的故事就此結束',
    '此處結束牛津學者的前言',
    '此處結束醫生的故事',
    '此處結束法典員的鄉農故事的前言',
    '此處結束托缽僧的故事',
    '此處結束商人的詹玉利故事',
    '此處結束喬叟的梅里彼和普魯登絲夫人的故事',
    '悲劇於此結束',
    '此處騎士打斷修道士的故事',
    '（此處前言結束)',
    '此處結束採購員的烏鴉故事',
]

exception_list = [
    '「講你的故事吧不必多說。」',
]

lines = [p.text.strip() for p in document.paragraphs] # remove spacing before and after

for i,line in enumerate(lines):
    if i < 6:
        continue
    if line == '':
        # if nothing in line skip
        continue
    if line[0].isdigit():
        # if line starts with number consider line chapter title
        chapter = line.split()[0]
        chinese_full_text[chapter] = []
        continue

    # skip section indicators
    if ('第' in line and '部份' in line):
        print('skipping 部份',line)
        continue
    if ('第' in line and '部分' in line):
        print('skipping 部分',line)
        continue
    if ('————' in line):
        print('skipping ----',line)
        continue
    if ('的故事' in line and ('————' in lines[i-1] or '————' in lines[i-2] or '————' in lines[i-3] or '————' in lines[i-4] or (lines[i-1]=='' and lines[i+1]=='')) and (line not in exception_list)):
        print('skipping 的故事',line)
        continue

    # skip all intro
    parts = line.split()
    text = parts[0]
    if text not in intro_ending_text_to_skip:
        chinese_full_text[chapter].append(text)

# previous_num = 0
# transitions = []
# for i,line in enumerate(full_text):
#     if line[0].isdigit():
#         transitions.append(i)

# insert N/A for missing chinese text
missing_tuple_after = [
    ('1.2','首先畫在牆上是一座森林，'),
    ('1.2','只要我能擁抱我情人在懷，'),
    ('1.2','號角裝滿了蜜酒，按習俗行事--'),
    ('1.3','為了對你的愛我到處忍受流汗。'),
    ('1.4','咱們老人家恐怕也變得那樣：'),
    ('1.5','衣著光鮮像林中的金絲/翅雀，'),
    ('1.5','這個樂活的學徒只好離去。'),
    ('2.1','如今她逃過所有的艱險危難，'),
    ('3.1','他說沒有什麼規律根據，'),
    ('3.1','但後來我卻痛心後悔；'),
    ('4.1','他們的主意便欲罷不能，'),
    ('4.1','而她懷著謙卑的心和輕鬆面容，'),
    ('5.1','像老鷹般亟欲翱翔天際，'),
    ('5.1','直像溫柔情人的舉動，'),
    ('5.2','之前，」她說，「我從不知道你的意思。'),
    ('6.2','他已殺了千人用這災疫，'),
    ('6.2','便不再記得他們要去找死神，'),
    ('8.1','有秘密事請益而且是好事一樁。'),# twice since two lines missing after this
    ('8.1','有秘密事請益而且是好事一樁。'),# twice since two lines missing after this
    ('8.3','我便羞愧得兩頰發燙。'),
]
for missing_tuple in missing_tuple_after:
    chapter, text = missing_tuple
    missing_text_insert_index = chinese_full_text[chapter].index(text) + 1
    chinese_full_text[chapter].insert(missing_text_insert_index,'N/A')

# truncate 4.1
chinese_full_text['4.1'] = chinese_full_text['4.1'][:1212]
# truncate 7.1, english is missing corresponding text
chinese_full_text['7.1'] = chinese_full_text['7.1'][:157]
# append 8.3 to 8.2 to match english website
chinese_full_text['8.2'] = chinese_full_text['8.2']+chinese_full_text['8.3']
# truncate 10.1 chinese
chinese_full_text['10.1'] = chinese_full_text['10.1'][:74]

skipping 部份 第一部份結束
skipping 部份 第二部份延續
skipping 部份 第二部份完
skipping ---- ————————-
skipping 部份 第三部份接續
skipping 部份 第三部份完
skipping ---- ———————————
skipping 部份 第四部份如下
skipping 的故事 騎士的故事就此結束
skipping 的故事 此處開始磨坊主的故事
skipping ---- ————————～～～
skipping 的故事 封邑管理員的故事
skipping 的故事 這裏開始封邑管理員的故事
skipping 的故事 封邑管理員的故事完
skipping ---- ————————————
skipping 的故事 廚師的故事
skipping 的故事 此處開始廚師的故事
skipping 部分 第一部分完
skipping 部分 第二部分接續
skipping 部分 第二部分完
skipping 部分 第三部分接續
skipping 的故事 律師的故事在此結束
skipping 的故事 律師的故事的尾聲
skipping 的故事 此處開始巴芙婦人的故事
skipping 的故事 此處結束巴芙婦人的故事
skipping 的故事 托缽僧的故事
skipping 的故事 此處開始托缽僧的故事
skipping 的故事 差役的故事
skipping 的故事 差役的故事在此開始
skipping 的故事 此處結束差役的故事。
skipping ---- —————————————
skipping 的故事 此處開始牛津學者的故事
skipping 部份 第一部份完
skipping 部份 第二部份開始
skipping 部份 第二部份結束
skipping 部份 第三部份開始
skipping 部份 第三部份結束
skipping 部份 第四部份開始
skipping 部分 第四部分結束
skipping 部分 第五部分接續
skipping 部份 『第六部份』
skipping 的故事 此處結束牛津學者的故事
skipping 的故事 商人的故事
skipping 的故事 這裡開始商人的故事
skipping 的故事 商人的故事尾聲
skipping 的故事 騎士侍從的故事
skipping 的故事 這

In [78]:
for chapter in chinese_full_text.keys():
    # print(chapter+':\t line counts:',len(full_text[chapter]))
    print(chapter+':\t line counts:',len(chinese_full_text[chapter]),chinese_full_text[chapter])

1.1:	 line counts: 860 ['時維四月，降下霏雨甘霖', '直透三月乾枯的根鬚', '每條根莖都沐浴在這潤液中', '如此能量誘發了春天花蕾；', '西風帶著它甜美的氣息', '吹過了雜叢草莽原野', '帶來了新芽嫩枝，青春的太陽', '只移轉到白羊宮一半的軌跡上，', '眾多小鳥忙著譜製旋律', '連在夜裡都張眼入眠', '(大自然如此挑動它們心靈)；', '民眾則渴望此時作朝聖行', '朝聖客打算出外訪尋異地', '到遠方著名的奉祀朝聖地；', '特別從英格蘭每個郡縣的角落', '人們都要到坎特伯里去', '拜謁在該地殉教的聖者', '在患病時曾助佑過他們。', '湊巧在這時節的某一天', '在南沃克塔巴德旅館我正歇著', '準備出發作朝聖之行', '一心虔敬要到坎特伯里去；', '夜裏，進來這旅館一行人', '合共有九又二十人之眾', '有各行各業人士；而因緣巧合', '他們結伴抵步，也都是朝聖客', '打算一起騎乘到坎特伯里去。', '此旅社房間充足馬廄又寬敞，', '我們大夥兒都妥被安頓接待；', '簡言之，到了太陽安歇時', '我已和每一位來客說過話', '旋即和他們建立友誼作伴', '大家約好翌日起個大清早', '取道上路，一如我所要敘述的。', '不過，話雖如此，趁有空閒，', '在故事開講之前，', '私意覺得按照情理', '要說一下所有的狀況', '就每個旅客的情況，', '是何許人，具何種身份地位', '又穿著何種衣飾現身等；', '且讓我先從一名騎士說起。', '有一名騎士，為人所尊敬', '從出道之初便矢志訂下了', '乘馬遊蕩時必得熱愛騎士精神:', '真誠、榮譽、慷慨和禮數。', '每次為君主作戰他都贏得尊敬', '他的騎乘遠屆，無人能及', '不論是在基督王國或異教之境', '所到之處備受尊崇、敬重，', '在亞歷山大城破時他就在場。', '在普魯士眾多參與國當中', '每次他都光榮地坐上首位，', '立陶宛他去出征，俄羅斯也一樣，', '同位階的基督徒沒有比他戰功更彪炳。', '在格瑞那達圍城時他參戰了，', '在阿格西拉，還騎乘至貝瑪利。', '在利厄斯和撒他利亞他參戰', '攻下城堡；而在地中海區域', '許多次大軍陣仗他都參與', '先後打過十五場殊死之戰，', '並為信仰在川姆森尼激戰', '三場，且殺敵

# Middle English and English from https://chaucer.fas.harvard.edu/pages/text-and-translations

In [5]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

def is_only_float_not_int(string):
    try:
        return divmod(float(string)*10,10)[1] > 0
    except ValueError:
        return False
    
def strip_digit(part):
    concatenated_words = ' '.join([word for word in part.split() if not word.isdigit()])
    return concatenated_words

def separate_lines(line):
    for character in '1234567890':
        line = line.replace(character,' ')
    parts = line.split('  ') # separate the middle english segments from english segments
    parts = [x.strip() for x in parts if x!='']
    middle_english_line, english_line = parts[::2],parts[1::2] # separate middle english lines from english lines
    return middle_english_line, english_line

def find_start_line_num(string):
    return int(string.split()[0])

def separate_middle_and_english(mixed_text):
    start_line_num = find_start_line_num(mixed_text)
    print('start_line_num is', start_line_num)
    mixed_text = mixed_text.replace('  ',' \t ') # use special character tab (' \t ') to delimit the middle english from english
    mixed_text = mixed_text.split(' ') # split via space all the words and numbers strings into items in a list
    start_search_index = 0
    while str(start_line_num) in mixed_text:
        # this will remove the number strings incrementally in ordered fashion 
        # so that out of sequence numbers that's part of the text do not get removed
        num_index = mixed_text[start_search_index:].index(str(start_line_num))
        num_index += start_search_index
        mixed_text[num_index]='\t'
        start_line_num += 1

    mixed_text = ' '.join(mixed_text).split('\t') # join string by space, separate by tab
    mixed_text = [x.strip() for x in mixed_text if x.strip() != ''] # remove all entries that are blank or pure tab
    
    # return the middle english from every 2 lines, english from every 2 lines starting from index=1
    return mixed_text[::2], mixed_text[1::2]

def remove_digits_at_end(chapter_text):
    for i,line in enumerate(chapter_text):
        while line[-1].isdigit():
            line=line[:-1]
        chapter_text[i] = line
    return chapter_text

urls = [
"https://chaucer.fas.harvard.edu/pages/general-prologue-0",
"https://chaucer.fas.harvard.edu/pages/knights-tale-0",
"https://chaucer.fas.harvard.edu/pages/millers-prologue-and-tale",
"https://chaucer.fas.harvard.edu/pages/reeves-prologue-and-tale",
"https://chaucer.fas.harvard.edu/pages/cooks-prologue-and-tale",
"https://chaucer.fas.harvard.edu/pages/man-laws-introduction-prologue-tale-and-epilogue",
"https://chaucer.fas.harvard.edu/pages/wife-baths-prologue-and-tale-0",
"https://chaucer.fas.harvard.edu/pages/friars-prologueand-tale",
"https://chaucer.fas.harvard.edu/pages/summoners-prologue-and-tale",
"https://chaucer.fas.harvard.edu/pages/clerks-prologue-tale-and-envoy",
"https://chaucer.fas.harvard.edu/pages/merchants-prologue-tale-and-epilogue",
"https://chaucer.fas.harvard.edu/pages/squires-introduction-and-tale",
"https://chaucer.fas.harvard.edu/pages/franklins-prologue-and-tale",
"https://chaucer.fas.harvard.edu/pages/physicians-tale-0",
"https://chaucer.fas.harvard.edu/pages/pardoners-prologue-introduction-and-tale",
"https://chaucer.fas.harvard.edu/pages/shipmans-tale-0",
"https://chaucer.fas.harvard.edu/pages/shipman-prioress-link",
"https://chaucer.fas.harvard.edu/pages/prioress-prologue-and-tale",
"https://chaucer.fas.harvard.edu/pages/prologue-and-tale-sir-thopas-and-hosts-interruption",
"https://chaucer.fas.harvard.edu/pages/tale-melibee-0",
"https://chaucer.fas.harvard.edu/pages/monks-prologue-and-tale",
"https://chaucer.fas.harvard.edu/pages/prologue-tale-and-epilogue-nuns-priest",
"https://chaucer.fas.harvard.edu/pages/second-nuns-prologue-and-tale",
"https://chaucer.fas.harvard.edu/pages/canons-yeomans-prologue-and-tale",
"https://chaucer.fas.harvard.edu/pages/manciples-prologue-and-tale",
"https://chaucer.fas.harvard.edu/pages/parsons-prologue-and-tale",
]


In [6]:
english_mixed_text = defaultdict(dict)
for url in urls:
    html = urlopen(url).read()
    soup = BeautifulSoup(html, features="html.parser")

    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    text = soup.get_text()
    lines = [line.replace(u'\xa0',u'') for line in text.splitlines()]
    # numbered_text = ''
    for line in lines:
        parts = line.split()
        if parts and is_only_float_not_int(parts[0]): # check if it is a number with decimal to use as chapter_num
            chapter_num = parts[0]
            english_mixed_text[chapter_num] = ''
        if parts and parts[0].isdigit():
            # numbered_text += line # this makes one long string for each chapter
            english_mixed_text[chapter_num] += ' '+line

In [7]:
english_full_text = defaultdict(dict)
middle_english_full_text = defaultdict(dict)

fix_english_mixed_text = defaultdict(dict)
fix_english_mixed_text['1.2']=[]
fix_english_mixed_text['1.2'].append(('134e','1345'))

fix_english_mixed_text['3.2']=[]
fix_english_mixed_text['3.2'].append(('But atte laste he seyde unto the wyf,','But atte laste he seyde unto the wyf,  N/A  ')) # it is missing english translation afterwards


for i in range(1271, 1286):
    # fix all the typo created by parsing
    fix_english_mixed_text['3.2'].append((str(i),' '+str(i)))

fix_english_mixed_text['7.5']=[]
# for some reason some line numbers are not used, artificially add them so they get taken out by the algorithm, one time fix
fix_english_mixed_text['7.5'].append(('until it were time that it must be known, and this may not be.','until it were time that it must be known, and this may not be.  1062  1063  '))
fix_english_mixed_text['7.5'].append(('when they see the punishing and chastising of the trespassers."','when they see the punishing and chastising of the trespassers."  1433  1434  '))
for i in range(967, 1889):
    # for some reason some numbers have 'A' attached
    fix_english_mixed_text['7.5'].append((str(i)+'A',' '))
    fix_english_mixed_text['7.5'].append((str(i)+'B',' '))
    fix_english_mixed_text['7.5'].append((str(i)+'C',' '))

fix_english_mixed_text['8.1']=[]
numeric_lines = ['177',
 '305',
 '326',
 '346',
 '356',
 '394',
 '412',
 '430',
 '457',
 '500',
 '539',
 '133',
 '186',
 '312',
 '343',
 '352',
 '374',
 '403',
 '419',
 '439',
 '467',
 '521']
for line in numeric_lines:
    fix_english_mixed_text['8.1'].append((line, ''))

fix_english_mixed_text['8.2']=[]
numeric_lines = ['1164', '1250', '1324', '1384', '1432', '1231', '1316', '1365', '1390']
for line in numeric_lines:
    fix_english_mixed_text['8.2'].append((line, ''))

fix_english_mixed_text['8.2']=[]
fix_english_mixed_text['8.2'].append(('And tell forth my tale of the canon','1020  N/A  And tell forth my tale of the canon')) # it is missing middle english before the english

for key in english_mixed_text.keys():
    numbered_text = english_mixed_text[key]
    for item in fix_english_mixed_text[key]:
        numbered_text=numbered_text.replace(item[0],item[1])
    m,e = separate_middle_and_english(numbered_text)
    middle_english_full_text[key] = m
    english_full_text[key] = e

# check 4.1
english_full_text['4.1'][1211] = ' '.join(english_full_text['4.1'][1211].split()[:-1]) # removes the last number at the end due to repeated line numbers
english_full_text['4.1'] = english_full_text['4.1'][:1212]
middle_english_full_text['4.1'] = middle_english_full_text['4.1'][:1212]
# fixes for 5.2
# ending string has numbers
english_full_text['5.2'] = remove_digits_at_end(english_full_text['5.2'])
# truncate chapter due to repeated text
english_full_text['5.2'] = english_full_text['5.2'][:916]
middle_english_full_text['5.2'] = middle_english_full_text['5.2'][:916]
# fixes for 8.1
english_full_text['8.1'] = remove_digits_at_end(english_full_text['8.1'])
middle_english_full_text['8.1'] = remove_digits_at_end(middle_english_full_text['8.1'])
# # fixes for 8.2
# english_full_text['8.2'] = remove_digits_at_end(english_full_text['8.2'])
# middle_english_full_text['8.2'] = remove_digits_at_end(middle_english_full_text['8.2'])


start_line_num is 1
start_line_num is 859
start_line_num is 3109
start_line_num is 3855
start_line_num is 4325
start_line_num is 1
start_line_num is 1
start_line_num is 1265
start_line_num is 1665
start_line_num is 1
start_line_num is 1213
start_line_num is 1
start_line_num is 709
start_line_num is 1
start_line_num is 287
start_line_num is 1
start_line_num is 435
start_line_num is 453
start_line_num is 691
start_line_num is 967
start_line_num is 1889
start_line_num is 2767
start_line_num is 1
start_line_num is 554
start_line_num is 1
start_line_num is 1


# Render aligned

In [8]:
chinese_full_text.keys()

dict_keys(['1.1', '1.2', '1.3', '1.4', '1.5', '2.1', '3.1', '3.2', '3.3', '4.1', '4.2', '5.1', '5.2', '6.1', '6.2', '7.1', '7.2', '7.3', '7.4', '7.5', '7.6', '7.7', '8.1', '8.2', '8.3', '9.1', '10.1'])

In [24]:
for chapter in middle_english_full_text.keys():
    chinese_count = len(chinese_full_text[chapter])
    middle_english_count = len(middle_english_full_text[chapter])
    english_count = len(english_full_text[chapter])
    print(chapter+':\t chinese english delta:',middle_english_count-chinese_count, 'middle-english delta:',english_count-middle_english_count)
print('\n')

1.1:	 chinese english delta: 0 middle-english delta: 0
1.2:	 chinese english delta: 0 middle-english delta: 0
1.3:	 chinese english delta: 0 middle-english delta: 0
1.4:	 chinese english delta: 0 middle-english delta: 0
1.5:	 chinese english delta: 0 middle-english delta: 0
2.1:	 chinese english delta: 0 middle-english delta: 0
3.1:	 chinese english delta: 0 middle-english delta: 0
3.2:	 chinese english delta: 0 middle-english delta: 0
3.3:	 chinese english delta: 0 middle-english delta: 0
4.1:	 chinese english delta: 0 middle-english delta: 0
4.2:	 chinese english delta: 0 middle-english delta: 0
5.1:	 chinese english delta: 0 middle-english delta: 0
5.2:	 chinese english delta: 0 middle-english delta: 0
6.1:	 chinese english delta: 0 middle-english delta: 0
6.2:	 chinese english delta: 0 middle-english delta: 0
7.1:	 chinese english delta: 0 middle-english delta: 0
7.2:	 chinese english delta: 0 middle-english delta: 0
7.3:	 chinese english delta: 0 middle-english delta: 0
7.4:	 chin

In [25]:
# for dictionary in [chinese_full_text, middle_english_full_text, english_full_text]:
#     for chapter in dictionary.keys():
#         # print(chapter+':\t line counts:',len(full_text[chapter]))
#         print(chapter+':\t line counts:',len(dictionary[chapter]),dictionary[chapter])
#     print('\n')

# check keys that are in common besides 8.2+8.3

In [26]:
chapters = list(english_mixed_text.keys())
print(chapters)

['1.1', '1.2', '1.3', '1.4', '1.5', '2.1', '3.1', '3.2', '3.3', '4.1', '4.2', '5.1', '5.2', '6.1', '6.2', '7.1', '7.2', '7.3', '7.4', '7.5', '7.6', '7.7', '8.1', '8.2', '9.1', '10.1']


In [27]:
import pandas as pd

align_dict = defaultdict()
for chapter in chapters:
    len_c, len_m, len_e = len(chinese_full_text[chapter]), len(middle_english_full_text[chapter]), len(english_full_text[chapter])
    pad_len = max(len_c, max(len_m,len_e))
    df = pd.DataFrame()
    df['chinese'] = chinese_full_text[chapter]+['N/A']*(pad_len-len_c)
    df['middle_english'] = middle_english_full_text[chapter]+['N/A']*(pad_len-len_m)
    df['english'] = english_full_text[chapter]+['N/A']*(pad_len-len_e)
    align_dict[chapter] = df



In [28]:
for index,line in align_dict['10.1'].iterrows():
    print(line['chinese']+' | '+line['english']+' | '+line['middle_english'])

採購員結束他故事之時， | By the time that the Manciple had his tale all ended, | By that the Maunciple hadde his tale al ended,
太陽從子午線上已下降 | The sun from the meridian was descended | The sonne fro the south lyne was descended
得很低，依我的視線而言， | So low that he was not, to my sight, | So lowe that he nas nat, to my sighte,
高度不超過二十九度。 | More than nine and twenty degrees in altitude. | Degrees nyne and twenty as in highte.
這時刻是四點鐘，我認為， | Four of the clock it was then, as I suppose, | Foure of the clokke it was tho, as I gesse,
因為這光景，或多或少， | For eleven feet, or little more or less, | For ellevene foot, or litel moore or lesse,
我的影子全長十一呎， | My shadow was there at that same time, | My shadwe was at thilke tyme, as there
這長度就像是我身高 | Of such feet as if my height were divided | Of swiche feet as my lengthe parted were
六呎分作相同等比的尺碼。 | Into six feet equal in size. | In sixe feet equal of proporcioun.
這時月亮升了上來—— | Therewith the moon's exaltation -- | Therwith the moones exaltacioun --
我說的是天秤座——緩緩上升 | I mean Libra 

# fixing individual chapters